In [1]:
import pyspark
from pyspark.sql import SparkSession
spark =SparkSession.builder.appName('agriculture').getOrCreate()

In [2]:
apy=spark.read.csv('C://Users/VarunNagendra/Desktop/training/assesments/mini_projects/agriculture/apy.csv',header=True,inferSchema=True)
area_cult=spark.read.csv('C://Users/VarunNagendra/Desktop/training/assesments/mini_projects/agriculture/area_cult.csv',header=True,inferSchema=True)
crops_price=spark.read.csv('C://Users/VarunNagendra/Desktop/training/assesments/mini_projects/agriculture/crops_price.csv',header=True,inferSchema=True)
rainfall_cleaned=spark.read.csv('C://Users/VarunNagendra/Desktop/training/assesments/mini_projects/agriculture/rainfall_cleaned.csv',header=True,inferSchema=True)
suicides=spark.read.csv('C://Users/VarunNagendra/Desktop/training/assesments/mini_projects/agriculture/suicides_10-14.csv',header=True,inferSchema=True)
data_set=spark.read.csv('C://Users/VarunNagendra/Desktop/training/assesments/mini_projects/agriculture/data_set.csv',header=True,inferSchema=True)

# spark sql

In [3]:
apy.createOrReplaceTempView("ap")
area_cult.createOrReplaceTempView("area")
crops_price.createOrReplaceTempView("cropprice")
rainfall_cleaned.createOrReplaceTempView("rain")
suicides.createOrReplaceTempView("suicide")
data_set.createOrReplaceTempView("data")

#### 1.Finding trends in crops in terms of production, area

In [4]:
spark.sql('''
select crop,round(sum(area)) as Total_Area,round(sum(Production)) as Total_production 
from ap 
group by crop
''').show()

+--------------------+-----------+----------------+
|                crop| Total_Area|Total_production|
+--------------------+-----------+----------------+
|            Arecanut|  6012013.0|     2.0346592E7|
|              Papaya|   133044.0|       3922521.0|
|        Black pepper|  3446555.0|       1784234.0|
|             Sesamum|2.8099154E7|     1.1009031E7|
|Beans & Mutter(Ve...|    48663.0|        211359.0|
|  Other  Rabi pulses|  8577095.0|       4805261.0|
|           Ash Gourd|     1644.0|             0.0|
|           Safflower|  5034059.0|       2821106.0|
|           Pineapple|   162760.0|       2127594.0|
|                Urad|4.7575718E7|     2.2410491E7|
|                 Tea|    77810.0|        135981.0|
|               Korra|   179309.0|        123713.0|
|       Cashewnut Raw|   146009.0|         98462.0|
|              Orange|   205201.0|       2338803.0|
|     Cond-spcs other|     2767.0|          2260.0|
|              Turnip|      802.0|           363.0|
|           

#### 2.State wise suicide rate of the farmers

In [5]:
#spark.sql('select state,(y2010+y2011+y2012+y2013+y2014)/100 as suicide_rate from suicide').show()
spark.sql('''
select states,sum(suicides)/100 as suicide_rate 
from data 
group by states 
order by states
''').show()

+--------------------+------------+
|              states|suicide_rate|
+--------------------+------------+
|ANDAMAN & NICOBAR...|         0.0|
|      ANDHRA PRADESH|     4253.28|
|   ARUNACHAL PRADESH|         7.9|
|               ASSAM|         0.0|
|               BIHAR|      109.29|
|          CHANDIGARH|         0.0|
|        CHHATTISGARH|      665.13|
|DADRA AND NAGAR H...|         0.0|
|                 GOA|        1.17|
|             GUJARAT|      477.21|
|             HARYANA|      315.81|
|    HIMACHAL PRADESH|       18.91|
|     JAMMU & KASHMIR|        4.74|
|           JHARKHAND|       12.11|
|           KARNATAKA|     3655.82|
|              KERALA|       930.7|
|      MADHYA PRADESH|     1989.15|
|         MAHARASHTRA|     3554.98|
|             MANIPUR|         0.0|
|           MEGHALAYA|       15.17|
+--------------------+------------+
only showing top 20 rows



#### 3.Which year has Highest cultivation of Pulses 

In [6]:
spark.sql('''
select year 
from area 
order by pulsestotal 
desc limit 1
''').show()

+----+
|year|
+----+
|2007|
+----+



#### 4.Which year has Highest cultivation among rice, wheat and coffee

In [7]:
#spark.sql('select year,sum(production) from area,ap where area.year=ap.Crop_year group by year').show(40)
spark.sql('''
select Crop_year 
from ap 
where crop in ('Rice','Wheat','Rice') 
group by Crop_year 
order by sum(production) 
limit 1
''').show(40)

+---------+
|Crop_year|
+---------+
|     2015|
+---------+



#### 5.Which state yields highest crop per year

In [8]:
#spark.sql('select State_Name,sum(Production)/Count(distinct(Crop_Year)) as production_per_year from ap group by State_Name order by State_Name').show()
spark.sql('''
select Crop_year,State_Name,productions from(
select Crop_year,State_Name,max(Production) as productions,
dense_rank() over(partition by Crop_year order by max(Production) desc) as row_count 
from ap group by Crop_year,State_Name) 
where row_count=1 
order by Crop_year
''').show()

+---------+----------+-----------+
|Crop_year|State_Name|productions|
+---------+----------+-----------+
|     1997|Tamil Nadu|   3.7158E7|
|     1998|    Kerala|     9.99E8|
|     1999|    Kerala|    1.059E9|
|     2000|    Kerala|     9.03E8|
|     2001|    Kerala|     8.95E8|
|     2002|    Kerala|     8.88E8|
|     2003|    Kerala|     8.98E8|
|     2004|    Kerala|      9.0E8|
|     2005|    Kerala|     8.63E8|
|     2006|    Kerala|     8.59E8|
|     2007|    Kerala|     8.51E8|
|     2008|    Kerala|     9.52E8|
|     2009|    Kerala|    1.063E9|
|     2010|    Kerala|  9.16053E8|
|     2011|Tamil Nadu|   1.2508E9|
|     2012|    Kerala|    1.125E9|
|     2013|Tamil Nadu|    1.212E9|
|     2014|    Kerala|    1.001E9|
|     2015|    Odisha|   465000.0|
+---------+----------+-----------+



#### 6.Finding how different factors like temperature affect production

In [9]:
spark.sql('''
select year,Avg_Temperature,avg(production) 
from data 
group by year,Avg_Temperature 
order by year
''').show()

+----+---------------+--------------------+
|year|Avg_Temperature|     avg(production)|
+----+---------------+--------------------+
|2000|           24.6|   9465958.230114356|
|2001|          24.73|   9848998.470013196|
|2002|           25.0|    7894312.98732308|
|2003|          24.72|   8239306.456534859|
|2004|          24.74|1.0594387703570504E7|
|2005|          24.58|1.0169099026219973E7|
|2006|          25.06|1.1423569899710525E7|
|2007|          24.77|   9160375.950173104|
|2008|          24.61| 1.027565699355526E7|
|2009|          25.11|1.1038175828256482E7|
|2010|          25.13|   8616951.537390713|
|2011|          24.66|2.1167017880369823E7|
|2012|          24.69|1.2807295840172412E7|
|2013|            0.0|1.9640165346849315E7|
|2014|            0.0|1.4735613318214286E7|
|2015|            0.0|   315230.2136363636|
+----+---------------+--------------------+



#### 7.Year wise suicide rates of farmers

In [10]:
spark.sql('''
select year,sum(suicides) as suicide_rate
from data 
group by year 
order by suicide_rate desc
''').show()

+----+------------+
|year|suicide_rate|
+----+------------+
|2010|    497780.0|
|2012|    437710.0|
|2011|    435725.0|
|2013|    341826.0|
|2014|    138294.0|
|2003|         0.0|
|2015|         0.0|
|2007|         0.0|
|2006|         0.0|
|2004|         0.0|
|2000|         0.0|
|2008|         0.0|
|2002|         0.0|
|2001|         0.0|
|2009|         0.0|
|2005|         0.0|
+----+------------+



#### 8.Which state receives highest rainfall in which year and what is the annual rainfall? 

In [11]:
spark.sql('''
select year,states,annual as maximumannual_rainfall 
from rain 
where annual=(select max(annual) from rain) 
group by year,states,annual
''').show()
#spark.sql('select year,states,max(annual) as maximumannual_rainfall from rain group by year,states order by max(annual) desc limit 1').show()
#spark.sql('select states,maxiannual,year from(select year,states,max(annual) as maxiannual,rank() over(partition by year order by max(annual) desc) as roww from rain group by year,states) where roww=1 order by year asc').show()

+----+--------------------+----------------------+
|year|              states|maximumannual_rainfall|
+----+--------------------+----------------------+
|2011|ANDAMAN & NICOBAR...|                3828.0|
+----+--------------------+----------------------+



#### 9.Crop wise cost of cultivation versus cost of production

In [12]:
spark.sql('''select Crop,sum(Area*Cost_per_Hectare) as cost_of_cultivation,sum(Price*Cost_per_quintal) as cost_of_production
from data 
group by crop 
order by  cost_of_cultivation desc ,cost_of_production desc
''').show()

+-------------------+--------------------+------------------+
|               Crop| cost_of_cultivation|cost_of_production|
+-------------------+--------------------+------------------+
|               Rice|5.880307326178279E12|      4.87442755E7|
|              Wheat|4.972782163515270...|     2.289999095E7|
|       Cotton(lint)| 2.45069003087245E12|      4.71064452E8|
|          Sugarcane| 2.28684043712397E12|432177.23720000003|
|          Groundnut|1.045283226719769...|      2.11948174E8|
|               Gram|7.978108300479102E11|     1.187512006E8|
|              Maize|6.264904585583301E11|     1.956743835E7|
|          Arhar/Tur|  5.6204538547138E11|       2.2856027E8|
|              Moong|   1.589241493702E11|     2.834757013E8|
|              Peach|                 0.0|               0.0|
|             Papaya|                 0.0|               0.0|
|       Black pepper|                 0.0|               0.0|
|                Tea|                 0.0|               0.0|
| Other 

#### 10.Highest crop production in particular season.

In [13]:
spark.sql('''
select season,max(production) as productions 
from ap 
group by season 
order by productions desc
''').show()

+-----------+-----------+
|     season|productions|
+-----------+-----------+
|Whole Year |   1.2508E9|
|Kharif     |1.7757963E7|
|Rabi       |  7556300.0|
|Winter     |  1841788.0|
|Summer     |   882289.0|
|Autumn     |   542692.0|
+-----------+-----------+

